In [ ]:
import yaml
import json

# Load files
with open("config_a.yml") as f:
    config = yaml.safe_load(f)

with open("Resonances.yml") as f:
    resonances = yaml.safe_load(f)

with open("final_params.json") as f:
    final_params = json.load(f)["value"]

# Kinematics
kinematics = {
    "initial_state": {
        "index": 0,
        "name": "Bp",
        "spin": str(config["particle"]["$top"]["Bp"]["J"]),
        "mass": config["particle"]["$top"]["Bp"]["mass"]
    },
    "final_state": [
        {
            "index": 1,
            "name": "D0",
            "spin": str(config["particle"]["$finals"]["D0"]["J"]),
            "mass": config["particle"]["$finals"]["D0"]["mass"]
        },
        {
            "index": 2,
            "name": "pi",
            "spin": str(config["particle"]["$finals"]["pi"]["J"]),
            "mass": config["particle"]["$finals"]["pi"]["mass"]
        },
        {
            "index": 3,
            "name": "D",
            "spin": str(config["particle"]["$finals"]["D"]["J"]),
            "mass": config["particle"]["$finals"]["D"]["mass"]
        }
        {
            "index": 4,
            "name": "K",
            "spin": str(config["particle"]["$finals"]["K"]["J"]),
            "mass": config["particle"]["$finals"]["K"]["mass"]
        }
    ]
}

# Chains from config_a and Resonances
chains = []
functions = []

for comb in ["DstD", "DK"]:
    for res_name in config["particle"][comb]:

        res_name_res = res_name
        if res_name in ["X0(2900)","X1(2900)"]:
            res_name_res = ["XDK","XDK2"][config["particle"]["DK"].index(res_name)]

        res_data = resonances[res_name_res]

        mass = final_params.get(f"{res_name_res}_mass", res_data.get("mass", res_data.get("m0", None)))
        width = final_params.get(f"{res_name_res}_width", res_data.get("width", res_data.get("g0", None)))

        if comb == "DstD":
            weight_real = final_params.get(f"Bp->{res_name}.K{res_name}->Dst.DDst->D0.pi_total_0r", 1.0)
            weight_imag = final_params.get(f"Bp->{res_name}.K{res_name}->Dst.DDst->D0.pi_total_0i", 0.0)
            weight = f"{weight_real} + {weight_imag} i"

        if comb == "DK":
            weight_real = final_params.get(f"Bp->{res_name}.Dst{res_name}->D.KDst->D0.pi_total_0r", 1.0)
            weight_imag = final_params.get(f"Bp->{res_name}.Dst{res_name}->D.KDst->D0.pi_total_0i", 0.0)
            weight = f"{weight_real} + {weight_imag} i"

        # Function-Definition
        func_name = f"BW_{res_name}"
        functions.append({
            "name": func_name,
            "type": "BreitWigner",
            "mass": str(mass),
            "width": str(width)
        })

        # Chain-Definition
        if comb == "DstD":
            chains.append({
                "name": res_name,
                "topology": [[[1, 2], 3], 4],  # DstD + K
                "propagators": [{
                    #"name": res_name,
                    "spin": str(res_data["J"]),
                    "node": [[1, 2], 3],
                    "parametrization": func_name
                }],
                "weight": weight,
                "vertices": [{
                    "type": "helicity",
                    "node": [[[1,2],3],4],
                    "helicities": []
                }]
            })

        if comb == "DK":
            chains.append({
                "name": res_name,
                "topology": [[1, 2], [3, 4]],  # DK + Dst
                "propagators": [{
                    "name": res_name,
                    "node": [2, 3],
                    "spin": str(res_data["J"]),
                    "parametrization": func_name
                }],
                "vertices": [{
                    "type": "helicity",
                    "node": [0, [2, 3]],
                    "helicities": []
                }],
                "weight": weight
            })

# JSON building
amp_json = {
    "kinematics": kinematics,
    "reference_topology": [[[1,2],3],4],
    "chains": chains,
    "functions": functions
}

with open("amplitude_serialization.json", "w") as f:
    json.dump(amp_json, f, indent=2)


In [3]:
all_res = config["particle"]["DK"]
DKs = ["XDK", "XDK2"]

print(["XDK","XDK2"][config["particle"]["DK"].index("X1(2900)")])

#print(final_params.get(f"Bp->{res_name}.Dst{res_name}->D.KDst->D0.pi_total_0r", 1.0))
#print(final_params.get(f"Bp->{res_name}.Dst{res_name}->D.KDst->D0.pi_total_0i", 0.0))
#print(res_name in resonances)

XDK2


In [3]:
import yaml
import json

# Load files
with open("config_a.yml") as f:
    config = yaml.safe_load(f)

with open("Resonances.yml") as f:
    resonances = yaml.safe_load(f)

with open("final_params.json") as f:
    final_params = json.load(f)["value"]

# --- Kinematics: jetzt mit 4 final states ---
kinematics = {
    "initial_state": {
        "index": 0,
        "name": "Bp",
        "mass": config["particle"]["$top"]["Bp"]["mass"],
        "spin": str(config["particle"]["$top"]["Bp"]["J"])
    },
    "final_state": [
        {"index": 1, "name": "D",  "mass": config["particle"]["$finals"]["D"]["mass"],  "spin": str(config["particle"]["$finals"]["D"]["J"])},
        {"index": 2, "name": "D0", "mass": config["particle"]["$finals"]["D0"]["mass"], "spin": str(config["particle"]["$finals"]["D0"]["J"])},
        {"index": 3, "name": "K",  "mass": config["particle"]["$finals"]["K"]["mass"],  "spin": str(config["particle"]["$finals"]["K"]["J"])},
        {"index": 4, "name": "pi", "mass": config["particle"]["$finals"]["pi"]["mass"], "spin": str(config["particle"]["$finals"]["pi"]["J"])}
    ]
}

chains = []
functions = []

# D* Breit-Wigner hinzufügen
functions.append({
    "name": "BW_Dst",
    "type": "BreitWigner",
    "mass": str(config["particle"]["Dst"]["mass"]),
    #"width": str(config["particle"]["Dst"]["width"])
})

# --- Hilfsfunktion für Resonanzketten ---
def add_chain(res_name, comb, node, topology):
    res_data = resonances[res_name]

    # Prüfen, ob NR (konstanter Term)
    is_constant = res_data.get("parametrization", "").startswith("Const")

    if not is_constant:
        # Mass/Width aus resonances oder fit params
        mass = final_params.get(f"{res_name}_mass", res_data.get("m0"))
        width = final_params.get(f"{res_name}_width", res_data.get("g0"))
    else:
        mass, width = None, None

    # Gewicht aus final_params
    if comb == "DstD":
        wr = final_params.get(f"Bp->{res_name}.K{res_name}->Dst.DDst->D0.pi_total_0r", 1.0)
        wi = final_params.get(f"Bp->{res_name}.K{res_name}->Dst.DDst->D0.pi_total_0i", 0.0)
    elif comb == "DK":
        wr = final_params.get(f"Bp->{res_name}.Dst{res_name}->D.KDst->D0.pi_total_0r", 1.0)
        wi = final_params.get(f"Bp->{res_name}.Dst{res_name}->D.KDst->D0.pi_total_0i", 0.0)
    else:
        wr, wi = 1.0, 0.0
    weight = f"{wr} + {wi} i"

    # Function-Definition
    func_name = f"{'Const' if is_constant else 'BW'}_{res_name}"
    func_entry = {"name": func_name}
    if is_constant:
        func_entry["type"] = "constant"
    else:
        func_entry.update({
            "type": "BreitWigner",
            "mass": str(mass),
            "width": str(width)
        })
    functions.append(func_entry)

    # Propagatoren: Resonanz + D*
    propagators = [{
        "name": res_name,
        "node": node,
        "spin": str(res_data["J"]),
        "parametrization": func_name
    }, {
        "name": "Dst",
        "node": [2,4],  # D0+π
        "spin": str(config["particle"]["Dst"]["J"]),
        "parametrization": "BW_Dst"
    }]

    vertices = [
        {"type": "helicity", "node": [0, node], "helicities": []},
        {"type": "helicity", "node": [[2,4],2], "helicities": []}
    ]

    chains.append({
        "name": res_name,
        "topology": topology,
        "propagators": propagators,
        "vertices": vertices,
        "weight": weight
    })

# --- Resonanzen im DstD-Kanal ---
for res_name in config["particle"]["DstD"]:
    add_chain(res_name, "DstD", [[2,4],1], [[[2,4],1],3])

# --- Resonanzen im DK-Kanal (z. B. X0, X1(2900)) ---
for res_name in config["particle"]["DK"]:
    add_chain(res_name, "DK", [2,3], [[2,3],1])

# JSON bauen
amp_json = {
    "kinematics": kinematics,
    "reference_topology": [[[2,4],1],3],
    "chains": chains,
    "functions": functions
}

with open("amplitude_serialization.json", "w") as f:
    json.dump(amp_json, f, indent=2)

print("✅ amplitude_serialization.json erzeugt (Resonanzen + NR).")


KeyError: 'X0(2900)'